# Q-4. 
```
Imagine you working as a sale manager now you need to predict the Revenue
and whether that particular revenue is on the weekend or not and find the
Informational_Duration using the Ensemble learning algorithm
Dataset This is the Dataset You can use this dataset for this question.
```

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("online_shoppers_intention.csv")

In [3]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [4]:
df.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')

In [5]:
df["Revenue"].value_counts()

False    10422
True      1908
Name: Revenue, dtype: int64

In [6]:
df["Weekend"].value_counts()

False    9462
True     2868
Name: Weekend, dtype: int64

In [7]:
len(df["Informational_Duration"].unique())

1258

In [8]:
X = df.drop(["Revenue", "Month"], axis=1)
y_revenue = df["Revenue"]
y_weekend = df["Weekend"]
y_informational_duration = df["Informational_Duration"]

In [9]:
cat_cols = list(df.select_dtypes(include="object"))
num_cols = list(df.select_dtypes(exclude="object"))

In [10]:
num_cols = ['Administrative', 'Administrative_Duration', 'Informational', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'OperatingSystems', 'Browser', 'Region', 'TrafficType']

In [11]:
cat_cols = ['VisitorType']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_revenue, y_test_revenue, y_train_weekend, y_test_weekend, y_train_info_dur, y_test_info_dur = train_test_split(
    X, y_revenue, y_weekend, y_informational_duration, test_size=0.2, random_state=42
)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder

revenue_col = ['Revenue']
weekend_col = ['Weekend']


cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

revenue_pipeline = Pipeline(steps=[
    ("encode", OrdinalEncoder())
])

weekend_pipeline = Pipeline(steps=[
    ("encode", OrdinalEncoder())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_cols),
    ("num", num_pipeline, num_cols)
])

revenue_preprocessor = ColumnTransformer([
    ("target", revenue_pipeline, revenue_col)
])

weekend_preprocessor = ColumnTransformer([
    ("target", weekend_pipeline, weekend_col)
])

In [14]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [15]:
y_train_processed_revenue = revenue_preprocessor.fit_transform(pd.DataFrame(y_train_revenue))
y_test_processed_revenue = revenue_preprocessor.transform(pd.DataFrame(y_test_revenue))

In [16]:
y_train_processed_weekend = weekend_preprocessor.fit_transform(pd.DataFrame(y_train_weekend))
y_test_processed_weekend = weekend_preprocessor.transform(pd.DataFrame(y_test_weekend))

In [17]:
# Basic Import
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn import svm

In [18]:
models = {
    'rfc': RandomForestClassifier(random_state=42, n_estimators=101, criterion="entropy"),
    "et": ExtraTreesClassifier(random_state=42, n_estimators=151),
    "gbc": GradientBoostingClassifier(random_state=42),
    "cat": CatBoostClassifier(random_state=42, verbose=0, learning_rate=0.01),
    "ada": AdaBoostClassifier(random_state=42),
    "light": LGBMClassifier(random_state=42),
}

macro_average_list = []
trained_models_list = []

# looping through dictionary, create model and evaluates it
for model in list(models.values()):
    model.fit(X_train_processed, y_train_processed_revenue)
    y_pred = model.predict(X_test_processed)
    report = classification_report(y_test_processed_revenue, y_pred, output_dict=True)    
    print(f"{model}:\n{classification_report(y_test_processed_revenue, y_pred)}")
    macro_average = report['macro avg']['f1-score']
    macro_average_list.append(macro_average)
    trained_models_list.append(model)

/var/folders/0x/fr5n585d50j8t9x54rbwy91w0000gn/T/ipykernel_26077/4210054742.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_processed, y_train_processed_revenue)


RandomForestClassifier(criterion='entropy', n_estimators=101, random_state=42):
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93      2055
         1.0       0.71      0.51      0.60       411

    accuracy                           0.88      2466
   macro avg       0.81      0.74      0.76      2466
weighted avg       0.88      0.88      0.88      2466



/var/folders/0x/fr5n585d50j8t9x54rbwy91w0000gn/T/ipykernel_26077/4210054742.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_processed, y_train_processed_revenue)


ExtraTreesClassifier(n_estimators=151, random_state=42):
              precision    recall  f1-score   support

         0.0       0.89      0.97      0.93      2055
         1.0       0.74      0.42      0.53       411

    accuracy                           0.88      2466
   macro avg       0.82      0.69      0.73      2466
weighted avg       0.87      0.88      0.86      2466



/Users/aneesaro/miniforge3/envs/pyc/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(random_state=42):
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93      2055
         1.0       0.72      0.53      0.61       411

    accuracy                           0.89      2466
   macro avg       0.81      0.75      0.77      2466
weighted avg       0.88      0.89      0.88      2466

<catboost.core.CatBoostClassifier object at 0x14b6c2520>:
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93      2055
         1.0       0.73      0.52      0.61       411

    accuracy                           0.89      2466
   macro avg       0.82      0.74      0.77      2466
weighted avg       0.88      0.89      0.88      2466



/Users/aneesaro/miniforge3/envs/pyc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(random_state=42):
              precision    recall  f1-score   support

         0.0       0.92      0.95      0.93      2055
         1.0       0.68      0.56      0.62       411

    accuracy                           0.88      2466
   macro avg       0.80      0.75      0.77      2466
weighted avg       0.88      0.88      0.88      2466



/Users/aneesaro/miniforge3/envs/pyc/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aneesaro/miniforge3/envs/pyc/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


LGBMClassifier(random_state=42):
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.94      2055
         1.0       0.73      0.55      0.62       411

    accuracy                           0.89      2466
   macro avg       0.82      0.75      0.78      2466
weighted avg       0.88      0.89      0.88      2466



In [19]:
macro_avg_max_value = max(macro_average_list)
macro_avg_max_index = macro_average_list.index(macro_avg_max_value)
best_model = trained_models_list[macro_avg_max_index]
best_model_name = list(models.keys())[macro_avg_max_index]

In [20]:
print(f"{best_model_name}: {round(macro_avg_max_value, 2)*100}")

light: 78.0
